In [1]:
%%time
import tensorflow as tf
import numpy as np
import pickle
import os
import json
import sys
import re
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.backends.backend_pdf
from textwrap import wrap
import utils
from rdflib import Graph
import copy
from datetime import datetime

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
gpu_options = tf.GPUOptions(allow_growth=True)
config = tf.ConfigProto(gpu_options=gpu_options,log_device_placement=True,allow_soft_placement=True)

working_path = '/data/bobby/AIDA/M18_copy/data/'
corpus_path = '/dvmm-filer2/projects/AIDA/data/ldc_isi_dryrun/dryrun/'

#rpi mention results in AIF
p_f = 'D1'
#grounding path
grounding_dict_path = working_path + 'tmp/backup/grounding_dict_'+p_f+'.pickle'


CPU times: user 2.65 s, sys: 1.11 s, total: 3.76 s
Wall time: 58.4 s


In [20]:
%%time
# loading
print('Loading AIF crawl dictionaries...')
with open(working_path + 'tmp/backup/entity2mention_dict_'+p_f+'.pickle', 'rb') as f:
    en2men = pickle.load(f)
# print("en2men:\n",list(en2men.keys())[:2])

ex_enlist = ['http://www.isi.edu/gaia/entities/e93d5c03-38fe-440b-821b-f890df901800', \
           'http://www.isi.edu/gaia/entities/b697db6d-7cb2-41d7-8b22-ea442c5cbada']
ex_en2men =  {k: en2men[k] for k in ex_enlist}
# print("en2men:\n",ex_en2men)

with open(working_path + 'tmp/backup/id2mentions_dict_'+p_f+'.pickle', 'rb') as f:
    id2men = pickle.load(f)
# print("id2men:\n",list(id2men.keys())[:2])

ex_idlist = ['HC0005KMS.mp4.ldcc', 'IC001K27C.ltf.xml']
ex_id2men =  {k: id2men[k] for k in ex_idlist}
# print("id2men:\n",ex_id2men)

# saving

with open(working_path + 'tmp/example_dicts/entity2mention_dict_'+p_f+'.pickle', 'wb') as f:
    pickle.dump(ex_en2men,f, protocol=pickle.HIGHEST_PROTOCOL)

with open(working_path + '/tmp/example_dicts/id2mentions_dict_'+p_f+'.pickle', 'wb') as f:
    pickle.dump(ex_id2men,f, protocol=pickle.HIGHEST_PROTOCOL)
print('Processing dicts done.')


Loading AIF crawl dictionaries...
Processing dicts done.
CPU times: user 832 ms, sys: 76 ms, total: 908 ms
Wall time: 951 ms


In [22]:
# print('Processing grounding_dict')


# with open(grounding_dict_path, 'rb') as f:
#     en_to_img_dict = pickle.load(f)
# # print("en_to_img_dict:\n",list(en_to_img_dict.keys())[:2]) 
# ex_grounded_enlist = ['http://www.isi.edu/gaia/entities/97c7e3ce-8e6d-4dba-a238-849a616386e7',\
#  'http://www.isi.edu/gaia/entities/86cebb5d-542e-4b1c-ab8e-6fd5b49f6936']
# ex_en_to_img_dict =  {k: en_to_img_dict[k] for k in ex_grounded_enlist}
# # print("en_to_img_dict:\n",ex_en_to_img_dict)


# with open(working_path + '/tmp/example_dicts/grounding_dict_'+p_f+'.pickle', 'wb') as f:
#     pickle.dump(ex_en_to_img_dict,f, protocol=pickle.HIGHEST_PROTOCOL)




with open(grounding_dict_path, 'rb') as f:
    en_to_img_dict = pickle.load(f)

#all grounded entities
grounded_en_dict = {}
s=0
for en in en_to_img_dict:
    for img_id in en_to_img_dict[en]['grounding']:
        if len(en_to_img_dict[en]['grounding'][img_id])>0:
            s+=1
            grounded_en_dict[en] = img_id
# print(s)    

# print("en_to_img_dict:\n",list(grounded_en_dict.keys())[:2]) 
ex_grounded_enlist = ['http://www.isi.edu/gaia/entities/42f9322f-8d09-45fc-988a-540da533e94b', \
'http://www.isi.edu/gaia/entities/c5649544-38d8-4e28-b104-af431556a1a1']
ex_en_to_img_dict =  {k: en_to_img_dict[k]['grounding'][grounded_en_dict[k]] for k in ex_grounded_enlist}
print("en_to_img_dict:\n",ex_en_to_img_dict)
# e= ex_en_to_img_dict['http://www.isi.edu/gaia/entities/42f9322f-8d09-45fc-988a-540da533e94b']
# print(len(e['grounding_features']))

print('\nProcessing grounding_dict done.')

en_to_img_dict:
 {'http://www.isi.edu/gaia/entities/42f9322f-8d09-45fc-988a-540da533e94b': {'bbox': [array([329,  78, 995, 387], dtype=int32)], 'bbox_norm': [array([0.2455224 , 0.14181818, 0.7425373 , 0.70363635], dtype=float32)], 'bbox_score': [0.754137], 'heatmap': [array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0

In [24]:
%%time
## Todo: merge result of USC
usc_f  = 'uscvision_grounding_output_cu_format'
usc_dict_path = working_path + 'usc_dict/'+usc_f+'.pickle'
with open(usc_dict_path, 'rb') as file:  
    data = pickle.load(file)

# usc_dict_path = working_path + 'usc_dict/'+usc_f+'.json'
# with open(usc_dict_path, 'r') as jsonfile:  
#     data = json.load(jsonfile)


# print(list(data.keys())[:1])
entity = data['http://www.isi.edu/gaia/entities/5ee94e37-3820-45d3-bf9c-bb16c117ac18']
print(entity)
# print(len(entity['grounding']['HC00001XQ.jpg.ldcc']['grounding_features'][0]))





{'grounding': {'HC00001XQ.jpg.ldcc': {'bbox': [array([ 12,  71, 258, 269], dtype=int32)], 'bbox_score': [0.9264336228000001], 'men_img_score': [1.0], 'grounding_features': [array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)], 'instance_features': [array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)], 'link_ids': [], 'link_scores': [], 'system': 'usc_vision'}}}
1024
CPU times: user 188 ms, sys: 0 ns, total: 188 ms
Wall time: 190 ms
